## 297. Serialize and Deserialize Binary Tree
- Description:
  <blockquote>
    Serialization is the process of converting a data structure or object into a sequence of bits so that it can be stored in a file or memory buffer, or transmitted across a network connection link to be reconstructed later in the same or another computer environment.

    Design an algorithm to serialize and deserialize a binary tree. There is no restriction on how your serialization/deserialization algorithm should work. You just need to ensure that a binary tree can be serialized to a string and this string can be deserialized to the original tree structure.

    Clarification: The input/output format is the same as how LeetCode serializes a binary tree. You do not necessarily need to follow this format, so please be creative and come up with different approaches yourself.

    

    Example 1:

    Input: root = [1,2,3,null,null,4,5]
    Output: [1,2,3,null,null,4,5]

    Example 2:

    Input: root = []
    Output: []

    

    Constraints:

        The number of nodes in the tree is in the range [0, 104].
        -1000 <= Node.val <= 1000
  </blockquote>

- URL: [Problem_URL](https://leetcode.com/problems/serialize-and-deserialize-binary-tree/description/)

- Topics: Tree

- Difficulty: Hard / Medium

- Resources: example_resource_URL

### Solution 1, Optimized Preorder Depth First Search (DFS) using list to serialize

The reason preorder traversal is used in this serialization/deserialization solution is because it allows us to reconstruct the tree unambiguously when combined with marker values (like 'None') for missing children.

#### Why Preorder Works

In preorder (Root → Left → Right), you:

1. First encounter the root.
2. Then recursively process the left subtree.
3. Then recursively process the right subtree.

When you serialize using preorder and include explicit markers for null nodes, the structure of the tree is fully captured in the order of the tokens. During deserialization, you can reconstruct the tree in the exact same recursive structure, because:

 - You know the first value is the root.
 - The next values belong to the left subtree until you hit enough 'None' markers to complete it.
 - Then comes the right subtree.

This matches perfectly with a recursive deserializer that consumes tokens in the same order.


#### Why Inorder Usually Doesn't Work

In inorder (Left → Root → Right), the root is in the middle of its subtrees. Without additional information (like knowing which value is the root), you can’t split the sequence into left and right subtrees during deserialization — especially when nulls are involved.

Even with 'None' markers, the recursive structure isn’t naturally aligned: you don’t know where the root is until you’ve already passed part of the left subtree, making it hard to build top-down.

📝 Exception: If you also stored extra info (e.g., indices or used a stack-based approach), it might be possible — but it’s not straightforward or efficient like preorder.


#### What About Postorder?

Postorder (Left → Right → Root) can work, but only if you reverse the logic:

- Serialize in postorder (left, right, root).
- Deserialize by reading the token stream backwards (so you get root first, then right, then left).

So yes, postorder can work, but you’d need to:

- Reverse the list of tokens before deserializing, or
- Use an iterative stack-based approach.

It’s less natural for recursive reconstruction compared to preorder.

#### Key Insight

The critical requirement is:

You must know the root of any (sub)tree before its children during deserialization.


---

- Time Complexity: O(N)
- Space Complexity: O(N)

In [ ]:
# Definition for a binary tree node.
# class TreeNode(object):
#     def __init__(self, x):
#         self.val = x
#         self.left = None
#         self.right = None

class Codec:
    def serialize(self, root):
        """Encodes a tree to a single string.

        :type root: TreeNode
        :rtype: str
        """
        def rserialize(node):
            if node:
                vals.append(str(node.val))
                rserialize(node.left)
                rserialize(node.right)
            else:
                vals.append('None')
        
        vals = []
        rserialize(root)
        return ','.join(vals)

    def deserialize(self, data):
        """Decodes your encoded data to tree.

        :type data: str
        :rtype: TreeNode
        """
        def rdeserialize():
            val = next(vals)
            
            if val == 'None':
                return None
            
            node = TreeNode(int(val))
            node.left = rdeserialize()
            node.right = rdeserialize()
            
            return node
        
        vals = iter(data.split(','))
        return rdeserialize()
        

# Your Codec object will be instantiated and called as such:
# codec = Codec()
# codec.deserialize(codec.serialize(root))

### Solution 2, Serialize and Deserialize to comma separated string Preorder Depth First Search (DFS)
Solution description
- Time Complexity: O(N)
  - In both serialization and deserialization functions, we visit each node exactly once
- Space Complexity: O(N)
  - In both serialization and deserialization functions, we keep the entire tree, either at the beginning or at the end

In [ ]:
# Definition for a binary tree node.
# class TreeNode(object):
#     def __init__(self, x):
#         self.val = x
#         self.left = None
#         self.right = None

class Codec:

    def serialize(self, root):
        """Encodes a tree to a single string.
        
        :type root: TreeNode
        :rtype: str
        """
        def rserialize(root, string):
            """ a recursive helper function for the serialize() function."""
            # check base case
            if root is None:
                string += 'None,'
            else:
                string += str(root.val) + ','
                string = rserialize(root.left, string)
                string = rserialize(root.right, string)
            return string
        
        return rserialize(root, '')
        

    def deserialize(self, data):
        """Decodes your encoded data to tree.
        
        :type data: str
        :rtype: TreeNode
        """
        def rdeserialize(l):
            """ a recursive helper function for deserialization."""
            if l[0] == 'None':
                l.pop(0)
                return None
                
            root = TreeNode(l[0])
            l.pop(0)
            root.left = rdeserialize(l)
            root.right = rdeserialize(l)
            return root

        data_list = data.split(',')
        root = rdeserialize(data_list)
        return root
        

# Your Codec object will be instantiated and called as such:
# ser = Codec()
# deser = Codec()
# ans = deser.deserialize(ser.serialize(root))

### Solution 3, Iterative Level-Order (BFS) Serialization & Deserialization
Solution description
- Time Complexity: O(N)
- Space Complexity: O(N)

In [ ]:
# Definition for a binary tree node.
# class TreeNode(object):
#     def __init__(self, x):
#         self.val = x
#         self.left = None
#         self.right = None

class Codec:
    def serialize(self, root):
        """Encodes a tree to a single string.
        
        :type root: TreeNode
        :rtype: str
        """
        if not root:
            return ""
        
        vals = []
        queue = collections.deque([root])
        
        while queue:
            node = queue.popleft()
            vals.append(str(node.val) if node else '#')
            if node:
                queue.append(node.left)
                queue.append(node.right)
                
        return ','.join(vals)
            
    def deserialize(self, data):
        """Decodes your encoded data to tree.
        
        :type data: str
        :rtype: TreeNode
        """
        if not data:
            return None
        
        received = data.split(',')
        root = TreeNode(received[0])
        queue = collections.deque([root])
        i = 1
        
        while queue:
            node = queue.popleft()
            
            if received[i]!='#':
                node.left = TreeNode(received[i])
                queue.append(node.left)
            
            i += 1
            if received[i]!='#':
                node.right = TreeNode(received[i])
                queue.append(node.right)
            i += 1
        
        return root
        

# Your Codec object will be instantiated and called as such:
# codec = Codec()
# codec.deserialize(codec.serialize(root))